Getting Started with Google Generative AI using the Gen AI SDK

## Overview

The [Google Gen AI SDK](https://googleapis.github.io/python-genai/) provides a unified interface to Google's generative AI API services. This SDK simplifies the process of integrating generative AI capabilities into applications and services, enabling developers to leverage Google's advanced AI models for various tasks.

In this tutorial, you learn about the key features of the Google Gen AI SDK for Python to help you get started with Google generative AI services and models including Gemini. You will complete the following tasks:

- Install the Gen AI SDK
- Connect to an API service
- Send text prompts
- Send multimodal prompts
- Set system instruction
- Configure model parameters
- Configure safety filters
- Start a multi-turn chat
- Control generated output
- Generate content stream
- Send asynchronous requests
- Count tokens and compute tokens
- Use context caching
- Function calling
- Batch prediction
- Get text embeddings


In [ ]:
#installing google gen ai sdk
%pip install --upgrade --quiet google-genai pandas

In [ ]:
#use the gen ai SDK
import datetime

from google import genai
from google.genai.types import (
    CreateBatchJobConfig,
    CreateCachedContentConfig,
    EmbedContentConfig,
    FunctionDeclaration,
    GenerateContentConfig,
    HarmBlockThreshold,
    HarmCategory,
    Part,
    SafetySetting,
    Tool,
)

Connect to a Generative AI API service
Google Gen AI APIs and models including Gemini are available in the following two API services:

Google AI for Developers: Experiment, prototype, and deploy small projects.
Vertex AI: Build enterprise-ready projects on Google Cloud.
The Gen AI SDK provided an unified interface to these two API services. This notebook shows how to use the Gen AI SDK in Vertex AI.

Vertex AI
To start using Vertex AI, you must have a Google Cloud project and enable the Vertex AI API. Learn more about setting up a project and a development environment.



In [ ]:
import os

PROJECT_ID = "qwiklabs-gcp-03-ba9aa1dc4706"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "global")

client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

In [ ]:
#choose a model
MODEL_ID = "gemini-2.5-flash"  # @param {type: "string"}

Send text prompts
Use the generate_content method to generate responses to your prompts. You can pass text to generate_content, and use the .text property to get the text content of the response.

For more examples of prompt engineering, refer to this notebook.

In [ ]:
response = client.models.generate_content(
    model=MODEL_ID, contents="What's the largest planet in our solar system?"
)

print(response.text)

from IPython.display import Markdown, display

display(Markdown(response.text))

## Send multimodal prompts

You can include text, PDF documents, images, audio and video in your prompt requests and get text or code responses.

For more examples of multimodal use cases, refer to [this notebook](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/intro_multimodal_use_cases.ipynb).

In [ ]:
from PIL import Image
import requests

image = Image.open(
    requests.get(
        "https://storage.googleapis.com/cloud-samples-data/generative-ai/image/meal.png",
        stream=True,
    ).raw
)

response = client.models.generate_content(
    model=MODEL_ID,
    contents=[
        image,
        "Write a short and engaging blog post based on this picture.",
    ],
)

print(response.text)

In [ ]:
response = client.models.generate_content(
    model=MODEL_ID,
    contents=[
        Part.from_uri(
            file_uri="https://storage.googleapis.com/cloud-samples-data/generative-ai/image/meal.png",
            mime_type="image/png",
        ),
        "Write a short and engaging blog post based on this picture.",
    ],
)

print(response.text)

Set system instruction
System instructions allow you to steer the behavior of the model. By setting the system instruction, you are giving the model additional context to understand the task, provide more customized responses, and adhere to guidelines over the user interaction.

In [ ]:
system_instruction = """
  You are a helpful language translator.
  Your mission is to translate text in English to French.
"""

prompt = """
  User input: I like bagels.
  Answer:
"""

response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config=GenerateContentConfig(
        system_instruction=system_instruction,
    ),
)

print(response.text)

Configure model parameters
You can include parameter values in each call that you send to a model to control how the model generates a response. Learn more about experimenting with parameter values.

In [ ]:
response = client.models.generate_content(
    model=MODEL_ID,
    contents="Tell me how the internet works, but pretend I'm a puppy who only understands squeaky toys.",
    config=GenerateContentConfig(
        temperature=0.4,
        top_p=0.95,
        top_k=20,
        candidate_count=1,
        seed=5,
        stop_sequences=["STOP!"],
        presence_penalty=0.0,
        frequency_penalty=0.0,
    ),
)

print(response.text)

Configure safety filters
The Gemini API provides safety filters that you can adjust across multiple filter categories to restrict or allow certain types of content. You can use these filters to adjust what's appropriate for your use case. See the Configure safety filters page for details.

For more examples of safety filters, refer to this notebook.

In [ ]:
prompt = """
    Write a list of 2 disrespectful things that I might say to the universe after stubbing my toe in the dark.
"""

safety_settings = [
    SafetySetting(
        category=HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
        threshold=HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
    ),
    SafetySetting(
        category=HarmCategory.HARM_CATEGORY_HARASSMENT,
        threshold=HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
    ),
    SafetySetting(
        category=HarmCategory.HARM_CATEGORY_HATE_SPEECH,
        threshold=HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
    ),
    SafetySetting(
        category=HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT,
        threshold=HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
    ),
]

response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config=GenerateContentConfig(
        safety_settings=safety_settings,
    ),
)

print(response.text)

In [ ]:
print(response.candidates[0].safety_ratings)

Start a multi-turn chat
The Gemini API enables you to have freeform conversations across multiple turns.

In [ ]:
system_instruction = """
  You are an expert software developer and a helpful coding assistant.
  You are able to generate high-quality code in any programming language.
"""

chat = client.chats.create(
    model=MODEL_ID,
    config=GenerateContentConfig(
        system_instruction=system_instruction,
        temperature=0.5,
    ),
)

In [ ]:
response = chat.send_message("Write a function that checks if a year is a leap year.")

print(response.text)

In [ ]:
response = chat.send_message("Okay, write a unit test of the generated function.")

print(response.text)

Control generated output
The controlled generation capability in Gemini API allows you to constraint the model output to a structured format. You can provide the schemas as Pydantic Models or a JSON string.

For more examples of controlled generation, refer to this notebook.

In [ ]:
from pydantic import BaseModel


class Recipe(BaseModel):
    name: str
    description: str
    ingredients: list[str]


response = client.models.generate_content(
    model=MODEL_ID,
    contents="List a few popular cookie recipes and their ingredients.",
    config=GenerateContentConfig(
        response_mime_type="application/json",
        response_schema=Recipe,
    ),
)

print(response.text)

In [ ]:
import json

json_response = json.loads(response.text)
print(json.dumps(json_response, indent=2))

You also can define a response schema in a Python dictionary. You can only use the supported fields as listed below. All other fields are ignored.

enum
items
maxItems
nullable
properties
required
In this example, you instruct the model to analyze product review data, extract key entities, perform sentiment classification (multiple choices), provide additional explanation, and output the results in JSON format.

In [ ]:
response_schema = {
    "type": "ARRAY",
    "items": {
        "type": "ARRAY",
        "items": {
            "type": "OBJECT",
            "properties": {
                "rating": {"type": "INTEGER"},
                "flavor": {"type": "STRING"},
                "sentiment": {
                    "type": "STRING",
                    "enum": ["POSITIVE", "NEGATIVE", "NEUTRAL"],
                },
                "explanation": {"type": "STRING"},
            },
            "required": ["rating", "flavor", "sentiment", "explanation"],
        },
    },
}

prompt = """
  Analyze the following product reviews, output the sentiment classification and give an explanation.

  - "Absolutely loved it! Best ice cream I've ever had." Rating: 4, Flavor: Strawberry Cheesecake
  - "Quite good, but a bit too sweet for my taste." Rating: 1, Flavor: Mango Tango
"""

response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config=GenerateContentConfig(
        response_mime_type="application/json",
        response_schema=response_schema,
    ),
)

print(response.text)

Generate content stream
By default, the model returns a response after completing the entire generation process. You can also use generate_content_stream method to stream the response as it is being generated, and the model will return chunks of the response as soon as they are generated.



In [ ]:
for chunk in client.models.generate_content_stream(
    model=MODEL_ID,
    contents="Tell me a story about a lonely robot who finds friendship in a most unexpected place.",
):
    print(chunk.text)
    print("*****************")

Send asynchronous requests
You can send asynchronous requests using the client.aio module. This module exposes all the analogous async methods that are available on client.

For example, client.aio.models.generate_content is the async version of client.models.generate_content.



In [ ]:
response = await client.aio.models.generate_content(
    model=MODEL_ID,
    contents="Compose a song about the adventures of a time-traveling squirrel.",
)

print(response.text)

Count tokens and compute tokens
You can use count_tokens method to calculates the number of input tokens before sending a request to the Gemini API. See the List and count tokens page for more details.

In [ ]:
#count tokens
response = client.models.count_tokens(
    model=MODEL_ID,
    contents="What's the highest mountain in Africa?",
)

print(response)

In [ ]:
#compute tokens
response = client.models.compute_tokens(
    model=MODEL_ID,
    contents="What's the longest word in the English language?",
)

print(response)

Function calling
Function calling lets you provide a set of tools that it can use to respond to the user's prompt. You create a description of a function in your code, then pass that description to a language model in a request. The response from the model includes the name of a function that matches the description and the arguments to call it with.

For more examples of Function Calling, refer to this notebook.

In [ ]:
get_destination = FunctionDeclaration(
    name="get_destination",
    description="Get the destination that the user wants to go to",
    parameters={
        "type": "OBJECT",
        "properties": {
            "destination": {
                "type": "STRING",
                "description": "Destination that the user wants to go to",
            },
        },
    },
)

destination_tool = Tool(
    function_declarations=[get_destination],
)

response = client.models.generate_content(
    model=MODEL_ID,
    contents="I'd like to travel to Paris.",
    config=GenerateContentConfig(
        tools=[destination_tool],
        temperature=0,
    ),
)

response.candidates[0].content.parts[0].function_call

Use context caching
Context caching lets you to store frequently used input tokens in a dedicated cache and reference them for subsequent requests, eliminating the need to repeatedly pass the same set of tokens to a model.



In [ ]:
#create a cache 
system_instruction = """
  You are an expert researcher who has years of experience in conducting systematic literature surveys and meta-analyses of different topics.
  You pride yourself on incredible accuracy and attention to detail. You always stick to the facts in the sources provided, and never make up new facts.
  Now look at the research paper below, and answer the following questions in 1-2 sentences.
"""

pdf_parts = [
    Part.from_uri(
        file_uri="gs://cloud-samples-data/generative-ai/pdf/2312.11805v3.pdf",
        mime_type="application/pdf",
    ),
    Part.from_uri(
        file_uri="gs://cloud-samples-data/generative-ai/pdf/2403.05530.pdf",
        mime_type="application/pdf",
    ),
]

cached_content = client.caches.create(
    model="gemini-2.5-flash",
    config=CreateCachedContentConfig(
        system_instruction=system_instruction,
        contents=pdf_parts,
        ttl="3600s",
    ),
)

In [ ]:
#use a cache 
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="What is the research goal shared by these research papers?",
    config=GenerateContentConfig(
        cached_content=cached_content.name,
    ),
)

print(response.text)

In [ ]:
#delete a catche
client.caches.delete(name=cached_content.name)

Batch prediction
Different from getting online (synchronous) responses, where you are limited to one input request at a time, batch predictions for the Gemini API in Vertex AI allow you to send a large number of requests to Gemini in a single batch request. Then, the model responses asynchronously populate to your storage output location in Cloud Storage or BigQuery.

Batch predictions are generally more efficient and cost-effective than online predictions when processing a large number of inputs that are not latency sensitive.

Prepare batch inputs
The input for batch requests specifies the items to send to your model for prediction.

Batch requests for Gemini accept BigQuery storage sources and Cloud Storage sources. You can learn more about the batch input formats in the Batch text generation page.

This tutorial uses Cloud Storage as an example. The requirements for Cloud Storage input are:

File format: JSON Lines (JSONL)
Located in us-central1
Appropriate read permissions for the service account
Each request that you send to a model can include parameters that control how the model generates a response. Learn more about Gemini parameters in the Experiment with parameter values page.

This is one of the example requests in the input JSONL file batch_requests_for_multimodal_input_2.jsonl:

{"request":{"contents": [{"role": "user", "parts": [{"text": "List objects in this image."}, {"file_data": {"file_uri": "gs://cloud-samples-data/generative-ai/image/office-desk.jpeg", "mime_type": "image/jpeg"}}]}],"generationConfig":{"temperature": 0.4}}}

In [ ]:
INPUT_DATA = "gs://cloud-samples-data/generative-ai/batch/batch_requests_for_multimodal_input_2.jsonl"  # @param {type:"string"}

Prepare batch output location
When a batch prediction task completes, the output is stored in the location that you specified in your request.

The location is in the form of a Cloud Storage or BigQuery URI prefix, for example: gs://path/to/output/data or bq://projectId.bqDatasetId.

If not specified, gs://STAGING_BUCKET/gen-ai-batch-prediction will be used for Cloud Storage source and bq://PROJECT_ID.gen_ai_batch_prediction.predictions_TIMESTAMP will be used for BigQuery source.

This tutorial uses a Cloud Storage bucket as an example for the output location.

You can specify the URI of your Cloud Storage bucket in BUCKET_URI, or
if it is not specified, a new Cloud Storage bucket in the form of gs://PROJECT_ID-TIMESTAMP will be created for you.

In [ ]:
BUCKET_URI = "[your-cloud-storage-bucket]"  # @param {type:"string"}

if BUCKET_URI == "[your-cloud-storage-bucket]":
    TIMESTAMP = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
    BUCKET_URI = f"gs://{PROJECT_ID}-{TIMESTAMP}"

    ! gsutil mb -l {LOCATION} -p {PROJECT_ID} {BUCKET_URI}

Send a batch prediction request
To make a batch prediction request, you specify a source model ID, an input source and an output location where Vertex AI stores the batch prediction results.

To learn more, see the Batch prediction API page.

In [ ]:
batch_job = client.batches.create(
    model=MODEL_ID,
    src=INPUT_DATA,
    config=CreateBatchJobConfig(dest=BUCKET_URI),
)
batch_job.name

Print out the job status and other properties. You can also check the status in the Cloud Console at https://console.cloud.google.com/vertex-ai/batch-predictions

In [ ]:
batch_job = client.batches.get(name=batch_job.name)

In [ ]:
#optionally you can list all the batch prediction job to complete 
for job in client.batches.list():
    print(job.name, job.create_time, job.state)

Wait for the batch prediction job to complete
Depending on the number of input items that you submitted, a batch generation task can take some time to complete. You can use the following code to check the job status and wait for the job to complete.

In [ ]:
import time

# Refresh the job until complete
while batch_job.state == "JOB_STATE_RUNNING":
    time.sleep(5)
    batch_job = client.batches.get(name=batch_job.name)

# Check if the job succeeds
if batch_job.state == "JOB_STATE_SUCCEEDED":
    print("Job succeeded!")
else:
    print(f"Job failed: {batch_job.error}")

Retrieve batch prediction results
When a batch prediction task is complete, the output of the prediction is stored in the location that you specified in your request. It is also available in batch_job.dest.bigquery_uri or batch_job.dest.gcs_uri.

Example output:

{"status": "", "processed_time": "2024-11-13T14:04:28.376+00:00", "request": {"contents": [{"parts": [{"file_data": null, "text": "List objects in this image."}, {"file_data": {"file_uri": "gs://cloud-samples-data/generative-ai/image/gardening-tools.jpeg", "mime_type": "image/jpeg"}, "text": null}], "role": "user"}], "generationConfig": {"temperature": 0.4}}, "response": {"candidates": [{"avgLogprobs": -0.10394711927934126, "content": {"parts": [{"text": "Here's a list of the objects in the image:\n\n* **Watering can:** A green plastic watering can with a white rose head.\n* **Plant:** A small plant (possibly oregano) in a terracotta pot.\n* **Terracotta pots:** Two terracotta pots, one containing the plant and another empty, stacked on top of each other.\n* **Gardening gloves:** A pair of striped gardening gloves.\n* **Gardening tools:** A small trowel and a hand cultivator (hoe).  Both are green with black handles."}], "role": "model"}, "finishReason": "STOP"}], "modelVersion": "gemini-2.5-flash", "usageMetadata": {"candidatesTokenCount": 110, "promptTokenCount": 264, "totalTokenCount": 374}}}

In [ ]:
import fsspec
import pandas as pd

fs = fsspec.filesystem("gcs")

file_paths = fs.glob(f"{batch_job.dest.gcs_uri}/*/predictions.jsonl")

if batch_job.state == "JOB_STATE_SUCCEEDED":
    # Load the JSONL file into a DataFrame
    df = pd.read_json(f"gs://{file_paths[0]}", lines=True)

    display(df)

Get text embeddings
You can get text embeddings for a snippet of text by using embed_content method. All models produce an output with 768 dimensions by default. However, some models give users the option to choose an output dimensionality between 1 and 768. See Vertex AI text embeddings API for more details.

In [ ]:
TEXT_EMBEDDING_MODEL_ID = "gemini-embedding-001"  # @param {type: "string"}

In [ ]:
response = client.models.embed_content(
    model=TEXT_EMBEDDING_MODEL_ID,
    contents=[
        "How do I get a driver's license/learner's permit?",
        "How do I renew my driver's license?",
        "How do I change my address on my driver's license?",
    ],
    config=EmbedContentConfig(output_dimensionality=128),
)

print(response.embeddings)